In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper

Using TensorFlow backend.


Num GPUs Available: 1


In [2]:
dimension = (64,64)
channels = 5
optimizer = Adam( lr = 1e-5 )
tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true[:,:])
                           
def mixd(output):
    rate = tf.math.exp(output[0,:,:,0]) #A 
    s = tf.math.sigmoid(output[0,:,:,1])
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)), #E
     tfd.Poisson(rate=rate) #F 
     ]
    mixture = tfd.Mixture(
          cat=tfd.Categorical(probs=tf.stack([1-s, s],axis=-1)),#D
          components=components)
    
    return mixture
    
def testnet(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):


    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(10, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64


    output = Conv2D(2, (1, 1), activation=tf.exp)(up01)  # 1 x 64x64

    #output = tfp.layers.DistributionLambda(tfp.distributions.Poisson( batch_shape=(), event_shape=(), sample_shape=() ))(output)
    #poisson = tfp.distributions.Poisson
    #output = tfp.layers.IndependentPoisson(1)(output) 
    #output = tfp.layers.DistributionLambda(poisson)(output)
    #print(poisson)
    print(mixd(output))

    output = tfp.layers.DistributionLambda(mixd)(output)
    model = Model(inputs=inputs, outputs=output)
    return model

def provideData(flatten=False,dimension=dimension,batch_size=10,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

In [3]:
train_,test_ = provideData(batch_size=400,dimension=dimension)
t = Trainer(testnet,
                    NLL,
                    (train_,test_),
                    batch_size = 400,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae","accuracy"])

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 
tfp.distributions.Mixture("Mixture", batch_shape=[64, 64], event_shape=[], dtype=float32)
[Load file failed]  ./model_data/testnet_function/testnet_function64x64x5.h5
[Load file failed]  ./model_data/testnet_function/testnet_function64x64x5history.json
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 5)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 10)   460         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)   

In [4]:
t.fit(10)
model = t.model
#print(model(train_[0][0]))

ValueError: axes don't match array

In [ ]:
train, test = provideData(batch_size=25,dimension=dimension)

In [ ]:
X,Y = train[50]
pred = model(X)
label = Y
Xt,_=train[500]
print(pred)
#print(tf.argmax(pred.sample()))

In [ ]:
img = np.array((pred.mean()*255),dtype=np.uint8)
Image.fromarray(img)

In [ ]:
import numpy as np
from PIL import Image
test_arr = np.array(pred)
#print(pred.max())
#test_arr = ((test_arr / test_arr.max() ) * 255 ).astype(np.uint8)
#print(test_arr.min())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(pred.mean())
plt.hist(test_arr.ravel(), bins=256, range=(0, 255), fc='k', ec='k')

In [ ]:
img = (test_arr*255).astype(np.uint8)
Image.fromarray(img)